### Obtain

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('kc_house_data.csv')

In [3]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


### Scrub

In [4]:
df = df.drop(columns="id") # drop 'id' column
df.head(0)

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15


In [5]:
df.info() # check datatypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 20 columns):
date             21597 non-null object
price            21597 non-null float64
bedrooms         21597 non-null int64
bathrooms        21597 non-null float64
sqft_living      21597 non-null int64
sqft_lot         21597 non-null int64
floors           21597 non-null float64
waterfront       19221 non-null float64
view             21534 non-null float64
condition        21597 non-null int64
grade            21597 non-null int64
sqft_above       21597 non-null int64
sqft_basement    21597 non-null object
yr_built         21597 non-null int64
yr_renovated     17755 non-null float64
zipcode          21597 non-null int64
lat              21597 non-null float64
long             21597 non-null float64
sqft_living15    21597 non-null int64
sqft_lot15       21597 non-null int64
dtypes: float64(8), int64(10), object(2)
memory usage: 3.3+ MB


'date' may have to be converted to int64 somehow. Or made into another feature

floors needs conversion. why would there be non-integer floors?

sqft_basement needs conversion to float or integer

yr_renovated be made integer

#### Changing Datatypes

In [6]:
df.floors.describe()

count    21597.000000
mean         1.494096
std          0.539683
min          1.000000
25%          1.000000
50%          1.500000
75%          2.000000
max          3.500000
Name: floors, dtype: float64

In [7]:
df.floors.unique()

array([1. , 2. , 1.5, 3. , 2.5, 3.5])

there seems to be a reason for # floors. will keep

In [8]:
df.sqft_basement.describe()

count     21597
unique      304
top         0.0
freq      12826
Name: sqft_basement, dtype: object

In [26]:
true_objects = []
for val in df.sqft_basement.unique().tolist():
    try:
        float(val)
    except ValueError:
        true_objects.append(val)
print(true_objects)

[]


removing the '?' entry, so that sqft_basement can be integer

In [14]:
df = df[df.sqft_basement != '?']

In [15]:
df['sqft_basement'] = df.sqft_basement.astype(float)

In [16]:
df.sqft_basement.dtype

dtype('float64')

In [17]:
mod_list = []
for n in df.sqft_basement:
    p = float(n) % 1
    if p > 0:
        mod_list.append(p)
    else:
        continue
print(mod_list)        

[]


refuses integer. because of decimals?

In [18]:
# df.sqft_basement.unique()

In [19]:
df['sqft_basement'] = df.sqft_basement.astype('O')

In [20]:
df.sqft_basement.head()

0      0
1    400
2      0
3    910
4      0
Name: sqft_basement, dtype: object

In [21]:
df['sqft_basement'] = df.sqft_basement.astype('int64')

In [27]:
true_objects = []
for val in df.sqft_basement.unique().tolist():
    try:
        float(val)
    except ValueError:
        true_objects.append(val)
print(true_objects)

[]


In [28]:
df.sqft_basement.dtype

dtype('int64')

waterfront is mostly empty so drop it

In [29]:
df.waterfront[df.waterfront > 0].describe()

count    142.0
mean       1.0
std        0.0
min        1.0
25%        1.0
50%        1.0
75%        1.0
max        1.0
Name: waterfront, dtype: float64

In [30]:
df.waterfront[df.waterfront == 0].shape[0]

18662

In [31]:
df.waterfront.shape[0]

21143

In [32]:
df.drop(['waterfront'], axis=1, inplace=True);

yr_renovated is similar to waterfront. but will be kept as an example of a boolean

In [33]:
df.yr_renovated[df.yr_renovated > 0].describe()

count     723.000000
mean     1996.053942
std        15.430251
min      1934.000000
25%      1987.000000
50%      2000.000000
75%      2007.500000
max      2015.000000
Name: yr_renovated, dtype: float64

In [34]:
df.yr_renovated[df.yr_renovated == 0].shape[0]

16666

In [35]:
df.yr_renovated.shape[0]

21143

date (sold) likely contains little information

In [ ]:
df.drop(['date'], axis=1, inplace=True);

#### Convert Categorical data encoded as integers to String type

categoricals hiding as integers

In [ ]:
df.select_dtypes(include='int64').head()

are categorical vars: yr_built, zipcode

bedroom, condition, grade, are probably discrete numerical

yr_built could be converted to 'age', a numerical var

the rest are true integers

In [ ]:
df['zipcode'] = df.zipcode.astype(object)

categoricals hiding as floats

In [ ]:
df.select_dtypes(include='float64').head(0)

In [ ]:
df.view.unique()

In [ ]:
df.view.dtype

view can be made integer later

#### Handle nulls

In [ ]:
df.isna().sum()

In [ ]:
df.dropna(subset=['view'], inplace=True);

In [ ]:
df.view.isna().sum()

In [ ]:
df.yr_renovated.isna().sum()

In [ ]:
df.yr_renovated.describe()

In [ ]:
df.dropna(subset=['yr_renovated'], inplace=True);

In [ ]:
df.yr_renovated.isna().sum()

In [ ]:
df['view'] = df.view.astype('int64')

In [ ]:
df.view.dtype

In [ ]:
df.isnull().any()

#### _(minor)_ Feature engineering

converting 'yr_built' to 'age'

In [ ]:
df.yr_built.head()

In [ ]:
age_list = []
for year in df['yr_built']:
    n = 2019-year
    age_list.append(n)

In [ ]:
df['age'] = age_list

In [ ]:
df.age.head()

In [ ]:
df.drop(['yr_built'], axis=1, inplace=True);

converting yr_renovated to a boolean

In [ ]:
df.yr_renovated.head()

In [ ]:
df.yr_renovated[df.yr_renovated > 0].describe()

In [ ]:
df.yr_renovated[df.yr_renovated > 0].value_counts().sum()

there are only 722 years - not a lot of information. "Renovated or Not" is more useful information

In [ ]:
yr_list = []
for year in df.yr_renovated:
    if year > 0:
        yr_list.append(1)
    else:
        yr_list.append(0)

In [ ]:
df['renovated'] = yr_list

In [ ]:
df['renovated'].head()

In [ ]:
df.drop(['yr_renovated'], axis=1, inplace=True);

#### Multicollinearity

In [ ]:
# Set the style of the visualization
sns.set(style="white")

# Create a covariance matrix
corr = df.corr()

# Generate a mask the size of our covariance matrix
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(10, 10))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmin=-1, vmax=1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5});

##### Highest Correlations with Target

In [ ]:
# abs(df.corr().price) > .5

best predictors of price @ r>.5 : bathrooms, sqft_living, grade, sqft_above, sqft_living15; (r>.7 is too high a threshold for finding price predictors)

##### Feature selection (removal)

In [ ]:
# (abs(df.corr()) > .7)
# (abs(df.corr()) > .7).iloc[0:4]
# (abs(df.corr()) > .7).iloc[5:10]
# (abs(df.corr()) > .7).iloc[11:16]

considerably correlated features:

price vs sqft_living, bathrooms vs sqft_living, sqft_living vs grade, sqft_living vs sqft_above, sqft_living vs sqft_living15

grade vs sqft_above, grade vs sqft_living15, sqft_above vs grade, sqft_above vs sqft_living15

sqft_lot15 vs sqft_lot

best candidates for removal (occurances of multicollinearity):
    
sqft_living (5)

grade (4)

sqft_living15 (3)

sqft_lot15 (1)

In [ ]:
df.drop(['sqft_living','grade','sqft_living15','sqft_lot15'], axis=1, inplace=True)

In [ ]:
df.head(0)

#### Feature Scaling
is unnecessary for Multiple Linear Regression models. But here is an example

In [ ]:
# df.hist(figsize=(10,10));

sqft_above, sqft_basement, & sqft_lot are notably skwed

In [ ]:
np.log(df.sqft_above).hist();

In [ ]:
df['log_sqft_above'] = np.log(df.sqft_above);

In [ ]:
np.log(df.sqft_lot).hist();

In [ ]:
df['log_sqft_lot'] = np.log(df.sqft_lot);

In [ ]:
np.log(df.sqft_basement).hist(range=(0,10));

In [ ]:
df['log_sqft_basement'] = np.log(df.sqft_basement);

In [ ]:
print(df.log_sqft_basement.min(), df.log_sqft_basement.max())

In [ ]:
inf_clean = []
for n in df.log_sqft_basement:
    if n == float('-inf'):
        inf_clean.append(0)
    else:
        inf_clean.append(n)

In [ ]:
# inf_clean[0:14]

In [ ]:
# df['log_sqft_basement'].head(13)

In [ ]:
df['log_sqft_basement'] = inf_clean

In [ ]:
print(df.log_sqft_basement.min(), df.log_sqft_basement.max())

These 3 features have been Log-scaled:

In [ ]:
df.drop(['sqft_above','sqft_lot', 'sqft_basement'], axis=1, inplace=True)

They are now smaller (in range and magnitude), more normal, and more readable!

Could scale the target:

In [ ]:
df['price'].hist();

In [ ]:
df['price'].plot.box()

In [ ]:
np.log((df['price'])).plot.box();

In [ ]:
np.log((df['price'])).hist();

In [ ]:
# price is now log(price) ***
df['price'] = np.log((df['price']))

#### Categoricals / One-hot encoding

In [ ]:
df.select_dtypes(include='object').head()

zipcode is the only remaining categorical var.

It can be one-hot encoded

In [ ]:
zc = df['zipcode']

In [ ]:
zc = pd.get_dummies(df['zipcode'])

In [ ]:
zc.head()

In [ ]:
# drop zipcode column from df
df.drop(['zipcode'], axis=1, inplace=True);

In [ ]:
## to join zipcode dummies to original dataframe
# df = df.join(zc);

In [ ]:
# df.head()

### Explore

In [ ]:
# df.describe()

In [ ]:
# df.hist(figsize=(10,10));

'view' and 'log_sqft_basement' are full of 0's. They _could_ be made into booleans

In [ ]:
df.info();

#### Outliers

In [ ]:
str(df.price.dtype)

In [ ]:
def box_stats(col):
    Q_1 = col.quantile(.25)
    Q_3 = col.quantile(.75)
    IQR = Q_3 - Q_1
    box_min = Q_1-1.5*IQR
    box_max = Q_3+1.5*IQR
    print(box_min, box_max)

Outliers in the target: we could focus on homes rather than mansions

In [ ]:
# df['price'].describe()

In [ ]:
# df.price.max() - df.price.min()# spread

In [ ]:
# box_stats(df.price)

In [ ]:
df['price'].plot.box();

In [ ]:
df.price[df['price'] > 650000].count()

In [ ]:
df.price[df['price'] < 650000].count()

In [ ]:
## code to toss 4000 outliers, but that's a bit much
# df = df[df['price'] < 650000] 

In [ ]:
df.price[df['price'] < 650000].plot.box();

In [ ]:
df.price[df['price'] < 650000].hist();

price is fairly normal without outliers

##### Outliers in a predictor

In [ ]:
df['bathrooms'].hist();

In [ ]:
df['bathrooms'].plot.box();
# df.boxplot(column='bathrooms'); # synonym

In [ ]:
def box_stats(col):
    Q_1 = col.quantile(.25)
    Q_3 = col.quantile(.75)
    IQR = Q_3 - Q_1
    box_min = Q_1-1.5*IQR
    box_max = Q_3+1.5*IQR
    print(box_min, box_max)

In [ ]:
box_stats(df.bathrooms)

In [ ]:
(df[df['bathrooms'] > 4]).shape

In [ ]:
(df[df['bathrooms'] <= 4]).shape

In [ ]:
(df['bathrooms'][df['bathrooms'] <= 4]).hist();

In [ ]:
(df['bathrooms'][df['bathrooms'] <= 4]).plot.box();

In [ ]:
## to toss 200 outliers
# df = df[df['bathrooms'] <= 4]).shape

#### Other exploration: seaborn plots

In [ ]:
# x_vars_list = list(df.columns); x_vars.pop(0); print(x_vars_list)

In [ ]:
## Warning! takes several seconds
# sns.pairplot(data=df, x_vars=x_vars_list, y_vars=['price']);

In [ ]:
# sns.jointplot(x=df.sqft_basement, y=df.price);

In [ ]:
# sns.kdeplot(data=df.price, y=df.log_sqft_basement)

Checking X vs Y linearity via Scatterplots

In [ ]:
# sns.set()

In [ ]:
# sns.scatterplot(x="bedrooms", y="price", data=df);

In [ ]:
# for x_var in list(df.columns):
#     print(sns.scatterplot(x=x_var, y="price", data=df));

### Model

define Outcomes v Columns

In [ ]:
df.columns

In [ ]:
outcome = 'price'

In [ ]:
x_cols = list(df.columns); x_cols.pop(0); print(x_cols)

Statsmodels- Ordinary Least Squares Regression

In [ ]:
# import statsmodels.formula.api import smf # may need this
from statsmodels.formula.api import ols

In [ ]:
# fitting the model #OLS
predictors = "+".join(x_cols)
formula = outcome + "~" + predictors
model = ols(formula=formula, data=df).fit()

In [ ]:
model.summary() # results

#### Checks of Intra-multicollinearity

Another test for multicollinearity is the variance inflation factor. Typically, variables with a vif of 5 or greater (or more definitively 10 or greater) are displaying multicollinearity with other variables in the feature set.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
X = df[x_cols]

In [ ]:
# vif = [variance_inflation_factor(X.values.i) for i in range(X.shape[1])]
# list(zip(x_cols, vif))

#### Assumptions Necessary for Linear Regression

1) Linearity

2) Check for Normality of Predictors / Q-Q

In [ ]:
# plt.style.use('ggplot')

In [ ]:
# for column in df:
#     df[column].plot.hist(normed=True, label = column+' histogram')
#     df[column].plot.kde(label = column+' kde')
#     plt.legend()
#     plt.show()

In [ ]:
# for column in df:
#     df[column].plot.box()
#     plt.legend()
#     plt.show()

In [ ]:
import statsmodels.api as sm
import scipy.stats as stats

In [ ]:
fig = sm.graphics.qqplot(model.resid, dist=stats.norm, line='45', fit=True)

3) Homoscedasticity

In [ ]:
subset = df[df['price']<650000] # ***
print('Percent removed:',(len(df) - len(subset))/len(df))
outcome = 'price'
x_cols = ['bedrooms', 'bathrooms', 'floors', 'view', 'condition', 'lat', 'long', 'age', 'renovated', 'log_sqft_above', 'log_sqft_lot', 'log_sqft_basement']
predictors = '+'.join(x_cols)
formula = outcome + "~" + predictors
model = ols(formula=formula, data=subset).fit()
# model.summary()

In [ ]:
plt.scatter(model.predict(subset[x_cols]), model.resid)
plt.plot(model.predict(subset[x_cols]), [0 for i in range(len(subset))])

In [ ]:
# import scipy
# from scipy import stats

In [ ]:
## Kolmogorov-Smirnov test of Normality
# scipy.stats.kstest()